In [69]:
import pandas as pd
from transformers import pipeline, AutoTokenizer

In [70]:
%pip install transformers[torch]

python(86588) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


zsh:1: no matches found: transformers[torch]
Note: you may need to restart the kernel to use updated packages.


In [71]:
df_reviews = pd.read_json('../data/All_Beauty.jsonl', lines=True)
df_meta = pd.read_json('../data/meta_All_Beauty.jsonl', lines=True)

  0%|          | 0/2640 [05:57<?, ?it/s]


In [72]:
df = pd.merge(df_reviews, df_meta, on=["parent_asin"], how="left")

In [73]:
df = df.sample(frac=0.01, random_state=42)

In [74]:
df = df.drop(columns="timestamp")

In [75]:
df = df.drop(columns="bought_together")

In [76]:
df = df.rename(columns={"rating": "labels"})

In [77]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/Users/sinnocen/vscode/reco_system/recommendation_system/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [78]:
from datasets import Dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

# Apply the tokenizer to the DataFrame
hf_dataset = Dataset.from_pandas(df)
tokenized_datasets = hf_dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['text'])
tokenized_datasets.set_format('torch')
train_test_split = tokenized_datasets.train_test_split(test_size=0.7)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

Map: 100%|██████████| 7015/7015 [00:00<00:00, 10942.10 examples/s]


In [79]:
id2label = {1: "1", 2: "2", 3: "3", 4: "4", 5: "5"}
label2id = {"1": 1, "2": 2, "3": 3, "4": 4, "5": 5}

In [80]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=5, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

/Users/sinnocen/vscode/reco_system/recommendation_system/.venv/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [82]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

  4%|▍         | 101/2640 [04:35<1:55:17,  2.72s/it]


In [83]:
trainer.train()

 20%|██        | 132/660 [06:55<16:56,  1.93s/it]

{'eval_loss': 0.4588784873485565, 'eval_runtime': 174.7036, 'eval_samples_per_second': 28.11, 'eval_steps_per_second': 1.757, 'epoch': 1.0}


 40%|████      | 264/660 [10:52<10:07,  1.53s/it]  